<a href="https://colab.research.google.com/github/Mario-RJunior/calculadora-imoveis/blob/master/machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning

Agora é a etapa em que iremos gerar um modelo usando técnicas de Machine Learning para gerar um modelo que irá nos ajudar a prever o valor de aluguéis de imóveis da cidade de São Paulo. Para isso usaremos como base os arquivos csv gerados durante a [análise exploratória](https://github.com/Mario-RJunior/calculadora-imoveis/blob/master/analise_exploratoria.ipynb).

## 1) Importar os dados

In [16]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.metrics import r2_score

In [6]:
treino = pd.read_csv('https://raw.githubusercontent.com/Mario-RJunior/calculadora-imoveis/master/treino_preprocessado.csv')
teste = pd.read_csv('https://raw.githubusercontent.com/Mario-RJunior/calculadora-imoveis/master/teste_preprocessado.csv')

In [7]:
# Divisão para variáveis X e y
X_train = treino.drop(labels='preco', axis=1)
y_train = treino['preco']
X_test = teste.drop('preco', axis=1)
y_test = teste['preco']

## 2) Modelo Baseline

Inicialmente criaremos um modelo denominado de ***Baseline*** que servirá como parâmetro para comparar seu resultado com os dos outros modelos que iremos gerar. É importante ressaltar que para que um modelo seja considerado bom ele deve ser superior ao baseline.

Como modelo baseline usaremos um algorítmo de regressão linear.

In [8]:
# Criando o modelo de regressão linear
rl = LinearRegression()
rl.fit(X_train, y_train)
rl.score(X_test, y_test)

0.44902642400898723

Note que este modelo tem um score de aproximadamente 0.45, que é bem ruim. Agora, precisamos testar outros modelos para encontrar o melhor e mais adequado possível.

## 3) Testando outros modelos

### 3.1) K-Nearest Neighbors (KNN)

In [10]:
neigh = KNeighborsRegressor()
neigh.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [12]:
neigh.score(X_test, y_test)

0.396512595795555

### 3.2) Random Forest

In [14]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [15]:
rf.score(X_test, y_test)

0.42453704379810175

### 3.3) Adaboost

In [18]:
regr = AdaBoostRegressor()
regr.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
                  n_estimators=50, random_state=None)

In [19]:
regr.score(X_test, y_test)

0.40008418343010127